<a href="https://colab.research.google.com/github/Info-stats-ai/Forecasting_Using_XGboost_and_LLM/blob/main/Xgboost%2BLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Mount Google Drive
from google.colab import drive

# This will prompt you for authorization.
# Click the link, sign in to your Google account, copy the authorization code,
# and paste it back into the box in this cell.
drive.mount('/content/drive')

print("\n✅ Google Drive successfully mounted!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

✅ Google Drive successfully mounted!


In [2]:
# Install necessary libraries first
!pip install pdfplumber -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 101.3 MB/s eta 0:00:00


In [3]:
import os
import zipfile

In [4]:
# --- CONFIGURATION ---
# This path MUST point to the folder containing your UPLOADED ZIP FILES.
# Please double-check that this is the correct path.
pdf_zip_folder_path = "/content/drive/My Drive/homeguide_zipped" # <--- UPDATE THIS PATH if needed
# -------------------

# These are the temporary folders we will create in the Colab environment.
unzipped_pdfs_folder = "unzipped_pdfs"
text_output_folder = "extracted_text"

# Create the folders to work in.
os.makedirs(unzipped_pdfs_folder, exist_ok=True)
os.makedirs(text_output_folder, exist_ok=True)

print("\n✅ Drive mounted and folders are set up successfully.")
print(f"I will look for zip files in: '{pdf_zip_folder_path}'")


✅ Drive mounted and folders are set up successfully.
I will look for zip files in: '/content/drive/My Drive/homeguide_zipped'


In [5]:
print(f"--- Starting Step 3: Unzipping files from '{pdf_zip_folder_path}' ---")

if not os.path.exists(pdf_zip_folder_path):
    print(f"❌ ERROR: The folder '{pdf_zip_folder_path}' was not found.")
else:
    zip_files = [f for f in os.listdir(pdf_zip_folder_path) if f.lower().endswith(".zip")]
    if not zip_files:
        print(f"❌ ERROR: No .zip files were found in '{pdf_zip_folder_path}'.")
    else:
        print(f"Found {len(zip_files)} zip files to process...")
        for zip_filename in zip_files:
            print(f"   Unzipping '{zip_filename}'...")
            zip_path = os.path.join(pdf_zip_folder_path, zip_filename)
            try:
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(unzipped_pdfs_folder)
            except Exception as e:
                print(f"   ⚠️ Could not unzip '{zip_filename}'. Error: {e}")

        # Verify the result by counting the number of items in the output folder.
        # Note: This counts files AND any sub-folders that were created.
        total_items = len(os.listdir(unzipped_pdfs_folder))
        print(f"\n✅ Unzipping complete! The '{unzipped_pdfs_folder}' folder now contains {total_items} items.")

--- Starting Step 3: Unzipping files from '/content/drive/My Drive/homeguide_zipped' ---
Found 3 zip files to process...
   Unzipping 'Homeguide.zip'...
   Unzipping 'Homeguide-20250902T030504Z-1-002.zip'...
   Unzipping 'Homeguide-20250902T030504Z-1-001.zip'...

✅ Unzipping complete! The 'unzipped_pdfs' folder now contains 1 items.


In [6]:
# Step 4: Find All PDFs Recursively and Rename Them

import os
import json

# This is the top-level folder we need to search inside.
unzipped_pdfs_folder = "unzipped_pdfs"

print("\n--- Starting Step 4: Finding and renaming all PDFs ---")

# This list will store the full path to every PDF we find.
all_pdf_paths = []

# os.walk is a powerful function that goes through a directory and all its subdirectories.
# This is what makes our script robust to the nested folder structure.
for root, dirs, files in os.walk(unzipped_pdfs_folder):
    for file in files:
        if file.lower().endswith(".pdf"):
            # We save the full, correct path to each PDF.
            all_pdf_paths.append(os.path.join(root, file))

total_files_found = len(all_pdf_paths)
if total_files_found == 0:
    print("❌ ERROR: No PDF files were found even after searching all subfolders.")
    print("Please check the 'unzipped_pdfs' folder to see if the files are there.")
else:
    print(f"✅ Success! Found {total_files_found} PDF files inside the subfolders.")
    print("Proceeding to rename...")

    name_mapping = {}
    # This list will store the NEW paths of the renamed files for the next step.
    renamed_files_paths = []

    # Now, we loop through the full paths we found and rename each file.
    for i, old_path in enumerate(all_pdf_paths):
        try:
            # The new path will be in the same directory as the old one.
            new_path = os.path.join(os.path.dirname(old_path), f"{i:04d}.pdf")

            # We map the new simple name back to the original full path for our records.
            name_mapping[os.path.basename(new_path)] = old_path
            os.rename(old_path, new_path)
            renamed_files_paths.append(new_path)
        except Exception as e:
            print(f"   ⚠️ Could not rename '{os.path.basename(old_path)}'. Error: {e}")

    # Save our mapping key for future reference.
    with open('filename_mapping.json', 'w') as f:
        json.dump(name_mapping, f, indent=4)

    print(f"\n✅ Renaming complete! {len(name_mapping)} files are now in a simple format.")


--- Starting Step 4: Finding and renaming all PDFs ---
✅ Success! Found 900 PDF files inside the subfolders.
Proceeding to rename...

✅ Renaming complete! 900 files are now in a simple format.


In [7]:
# Step 5: Extract Text Using pdfplumber

import pdfplumber
import os

# These are the folders we are working with
text_output_folder = "extracted_text"

print("\n--- Starting Step 5: Extracting text with the upgraded pdfplumber library ---")

# This script relies on the 'renamed_files_paths' list created in the previous cell.
# This check ensures we have the list before we start.
if 'renamed_files_paths' not in locals() or not renamed_files_paths:
     print("❌ ERROR: Could not find the list of renamed files.")
     print("Please re-run the previous cell (Step 4) to generate the file list.")
else:
    total_files_to_extract = len(renamed_files_paths)
    print(f"Found {total_files_to_extract} cleaned PDF files to process...")

    # Loop through our list of clean, renamed PDF paths
    for i, pdf_path in enumerate(renamed_files_paths):
        # Get the simple filename (e.g., "0001.pdf") for progress messages
        pdf_filename = os.path.basename(pdf_path)

        # Print a progress update every 50 files to show it's working
        if (i + 1) % 50 == 0 or i == 0:
            print(f"Processing file {i+1}/{total_files_to_extract}: {pdf_filename}...")

        try:
            full_text = ""
            # Open the PDF with pdfplumber
            with pdfplumber.open(pdf_path) as pdf:
                # Loop through each page in the PDF
                for page in pdf.pages:
                    # .extract_text() is excellent at preserving the layout of tables.
                    # x_tolerance=2 helps in aligning text that is slightly offset.
                    text = page.extract_text(x_tolerance=2)
                    if text:
                        full_text += text + "\n\n--- Page Break ---\n\n"

            # Create the output filename (e.g., "0001.txt")
            text_filename = pdf_filename.replace('.pdf', '.txt')
            output_path = os.path.join(text_output_folder, text_filename)

            # Save the extracted text to the new file
            with open(output_path, "w", encoding="utf-8") as f:
                f.write(full_text)
        except Exception as e:
            # If one PDF is corrupted or unreadable, this will log the error and continue
            print(f"   ⚠️ Could not process {pdf_filename}. It may be corrupted or empty. Error: {e}")

    print(f"\n✅✅✅ Data Preparation Complete! ✅✅✅")
    print(f"High-quality text from all files has been extracted to the '{text_output_folder}' folder.")


--- Starting Step 5: Extracting text with the upgraded pdfplumber library ---
Found 900 cleaned PDF files to process...
Processing file 1/900: 0000.pdf...


Processing file 50/900: 0049.pdf...


Processing file 100/900: 0099.pdf...
Processing file 150/900: 0149.pdf...


Processing file 200/900: 0199.pdf...


Processing file 250/900: 0249.pdf...


Processing file 300/900: 0299.pdf...


Processing file 350/900: 0349.pdf...
Processing file 400/900: 0399.pdf...
Processing file 450/900: 0449.pdf...


Processing file 500/900: 0499.pdf...


Processing file 550/900: 0549.pdf...
Processing file 600/900: 0599.pdf...
Processing file 650/900: 0649.pdf...


Processing file 700/900: 0699.pdf...
Processing file 750/900: 0749.pdf...
Processing file 800/900: 0799.pdf...
Processing file 850/900: 0849.pdf...
Processing file 900/900: 0899.pdf...

✅✅✅ Data Preparation Complete! ✅✅✅
High-quality text from all files has been extracted to the 'extracted_text' folder.


In [10]:
import pandas as pd
import os

In [ ]:
# Install the Python library for Google's models
!pip install -q -U google-generativeai pandas


In [ ]:
import google.generativeai as genai
from google.colab import userdata
import os

In [ ]:
# Securely access the API key you stored in Colab's secrets.
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
    print("✅ Gemini API key configured successfully!")
except Exception as e:
    print("⚠️ Error configuring API key.")
    print("Please make sure you have created a secret named 'GEMINI_API_KEY' and given this notebook access to it.")

# Verify that the input folder from our previous work exists.
text_input_folder = "extracted_text"
if not os.path.exists(text_input_folder):
     print(f"❌ CRITICAL ERROR: The input folder '{text_input_folder}' was not found.")

In [13]:
# Phase 2 - Step 2: Define the AI Structuring Function

import json
import time

def get_structured_data_from_llm(text_content, filename):
    """
    Acts as our AI Analyst. It sends text to the Gemini model and asks it
    to extract structured data based on a specific prompt and JSON schema.

    Args:
        text_content (str): The raw text extracted from a PDF.
        filename (str): The original filename, for context.

    Returns:
        dict: A dictionary containing the extracted data, or None if an error occurs.
    """
    # This JSON schema is our instruction manual for the AI.
    # We are telling it exactly what fields we want and what type of data they should be.
    # This is flexible enough to handle both specific estimates and general guides.
    json_schema = {
        "type": "object",
        "properties": {
            "project_type": {"type": "string", "description": "A general category for the project, like 'Kitchen Remodel', 'Roofing Guide', or 'Interior Painting Estimate'."},
            "summary_description": {"type": "string", "description": "A concise, one-sentence summary of the document's main purpose or the scope of work."},
            "total_cost": {"type": "number", "description": "The final, total cost or grand total of the project. If it is a general guide or no total is found, this MUST be null."},
            "line_items": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "item_description": {"type": "string"},
                        "item_cost": {"type": "number"}
                    }
                },
                "description": "A list of individual work items and their costs. If none are found, this should be an empty list []."
            }
        },
        "required": ["project_type", "summary_description", "total_cost", "line_items"]
    }

    # We configure the model to always return its output in the JSON format we defined.
    generation_config = genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema=json_schema
    )
    model = genai.GenerativeModel('gemini-1.5-flash-latest', generation_config=generation_config)

    # This prompt is the direct instruction to our AI Analyst.
    prompt = f"""
    Analyze the following text from a document originally titled '{filename}'.
    Your task is to act as a data analyst and extract key information based on the schema.

    1.  Carefully determine the overall 'project_type'.
    2.  Write a brief, neutral 'summary_description' of the document's content.
    3.  Find the final 'total_cost'. It is CRITICAL that if no specific total project cost is listed, you return null for this field. Do not guess or sum up line items.
    4.  Extract any specific 'line_items' that have an associated cost. If the document is a general guide with no line items, return an empty list.

    Here is the text:
    ---
    {text_content[:20000]}
    ---
    """

    # This loop will retry the API call if it fails, which makes our script more robust.
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            return json.loads(response.text)
        except Exception as e:
            print(f"      - API Error: {e}. Retrying in {2**attempt}s...")
            time.sleep(2**attempt)

    print(f"      - Failed to get a valid response for {filename} after several retries.")
    return None

print("✅ AI Analyst function defined successfully.")
print("Ready to process the text files in the next step.")

✅ AI Analyst function defined successfully.
Ready to process the text files in the next step.


In [ ]:
# Phase 2 - Step 3 (The Definitive Robust Script)

import pandas as pd
import os
import time

# --- CONFIGURATION ---
text_input_folder = "extracted_text"
output_csv_path = "structured_data.csv"
start_from_file_number = 0 # If the script stops, you can set this to the last file number to resume
# -------------------

# --- THE FIX PART 1: A more patient retry function ---
# We are moving the retry logic into the function itself to make it more robust.
def get_structured_data_from_llm_patiently(text_content, filename):
    # This is the same schema and model setup as before
    json_schema = { "type": "object", "properties": { "project_type": {"type": "string"}, "summary_description": {"type": "string"}, "total_cost": {"type": "number"}, "line_items": { "type": "array", "items": { "type": "object", "properties": { "item_description": {"type": "string"}, "item_cost": {"type": "number"} } } } }, "required": ["project_type", "summary_description", "total_cost", "line_items"] }
    generation_config = genai.GenerationConfig( response_mime_type="application/json", response_schema=json_schema )
    model = genai.GenerativeModel('gemini-1.5-flash-latest', generation_config=generation_config)
    prompt = f"Analyze the following text from a document originally titled '{filename}'. Act as a data analyst. Extract the overall 'project_type', a brief 'summary_description', the final 'total_cost' (or null if not found), and a list of 'line_items' with costs (or an empty list). TEXT: --- {text_content[:20000]} ---"

    # We will use longer, more patient delays for retries
    retry_delays = [5, 10, 15] # Wait 5s, then 10s, then 15s
    for attempt, delay in enumerate(retry_delays):
        try:
            response = model.generate_content(prompt)
            return json.loads(response.text)
        except Exception as e:
            print(f"      - API Error on attempt {attempt+1}: {e}. Retrying in {delay}s...")
            time.sleep(delay)

    print(f"      - FAILED to get a valid response for {filename} after all retries.")
    return None
# ----------------------------------------------------


print(f"--- Starting the DEFINITIVE structuring process on the '{text_input_folder}' folder ---")
text_files = sorted([f for f in os.listdir(text_input_folder) if f.endswith(".txt")])
total_files = len(text_files)

if total_files > 0:
    all_structured_data = []
    print(f"Found {total_files} text files. This will be very slow but very reliable...")

    # Loop through every file, starting from the number you specify
    for i, filename in enumerate(text_files):
        # This is the resume logic
        if i < start_from_file_number:
            continue

        print(f"  Processing file {i+1}/{total_files}: {filename}...")
        filepath = os.path.join(text_input_folder, filename)
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()

        # Call our new, more patient function
        structured_data = get_structured_data_from_llm_patiently(content, filename)

        if structured_data:
            structured_data['source_file'] = filename
            all_structured_data.append(structured_data)
        else:
            print(f"   - Skipping {filename} as no structured data was returned.")

        # --- THE FIX PART 2: A much longer pause between files ---
        # We will wait 10 seconds between processing each file to be extremely safe.
        time.sleep(10)
        # --------------------------------------------------------

    if all_structured_data:
        print("\n✅ Processing complete! Compiling results into a CSV file...")
        df = pd.DataFrame(all_structured_data)
        df.to_csv(output_csv_path, index=False, encoding='utf-8')
        print(f"\n✅✅✅ Phase 2 Complete! ✅✅✅")
        print(f"Successfully created '{output_csv_path}' with {len(df)} rows.")
        display(df.head())

--- Starting the DEFINITIVE structuring process on the 'extracted_text' folder ---
Found 900 text files. This will be very slow but very reliable...
  Processing file 1/900: 0000.txt...
